In [ ]:
from math import log
from pymystem3 import Mystem
m = Mystem()
import re
import os
import collections
from collections import defaultdict, Counter
from flask import Flask, request, render_template
app = Flask(__name__)


k1 = 2.0
b = 0.75

def score_BM25(n, qf, N, dl, avdl):
    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    frac = ((k1 + 1) * qf) / (K + qf)
    return IDF * frac


def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

def search_words(query):
    stop_words = ['а', 'будем', 'будет', 'будете', 'будешь', 'буду', 'будут', 'будучи', 'будь', 'будьте', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вами', 'вас', 'весь', 'во', 'вот', 'все', 'всё', 'всего', 'всей', 'всем', 'всём', 'всеми', 'всему', 'всех', 'всею', 'всея', 'всю', 'вся', 'вы', 'да', 'для', 'до', 'его', 'едим', 'едят', 'ее', 'её', 'ей', 'ел', 'ела', 'ем', 'ему', 'емъ', 'если', 'ест', 'есть', 'ешь', 'еще', 'ещё', 'ею', 'же', 'за', 'и', 'из', 'или', 'им', 'ими', 'имъ', 'их', 'к', 'как', 'кем', 'ко', 'когда', 'кого', 'ком', 'кому', 'комья', 'которая', 'которого', 'которое', 'которой', 'котором', 'которому', 'которою', 'которую', 'которые', 'который', 'которым', 'которыми', 'которых', 'кто', 'меня', 'мне', 'мной', 'мною', 'мог', 'моги', 'могите', 'могла', 'могли', 'могло', 'могу', 'могут', 'мое', 'моё', 'моего', 'моей', 'моем', 'моём', 'моему', 'моею', 'можем', 'может', 'можете', 'можешь', 'мои', 'мой', 'моим', 'моими', 'моих', 'мочь', 'мою', 'моя', 'мы', 'на', 'нам', 'нами', 'нас', 'наса', 'наш', 'наша', 'наше', 'нашего', 'нашей', 'нашем', 'нашему', 'нашею', 'наши', 'нашим', 'нашими', 'наших', 'нашу', 'не', 'него', 'нее', 'неё', 'ней', 'нем', 'нём', 'нему', 'нет', 'нею', 'ним', 'ними', 'них', 'но', 'о', 'об', 'один', 'одна', 'одни', 'одним', 'одними', 'одних', 'одно', 'одного', 'одной', 'одном', 'одному', 'одною', 'одну', 'он', 'она', 'оне', 'они', 'оно', 'от', 'по', 'при', 'с', 'сам', 'сама', 'сами', 'самим', 'самими', 'самих', 'само', 'самого', 'самом', 'самому', 'саму', 'свое', 'своё', 'своего', 'своей', 'своем', 'своём', 'своему', 'своею', 'свои', 'свой', 'своим', 'своими', 'своих', 'свою', 'своя', 'себе', 'себя', 'собой', 'собою', 'та', 'так', 'такая', 'такие', 'таким', 'такими', 'таких', 'такого', 'такое', 'такой', 'таком', 'такому', 'такою', 'такую', 'те', 'тебе', 'тебя', 'тем', 'теми', 'тех', 'то', 'тобой', 'тобою', 'того', 'той', 'только', 'том', 'томах', 'тому', 'тот', 'тою', 'ту', 'ты', 'у', 'уже', 'чего', 'чем', 'чём', 'чему', 'что', 'чтобы', 'эта', 'эти', 'этим', 'этими', 'этих', 'это', 'этого', 'этой', 'этом', 'этому', 'этот', 'этою', 'эту', 'я']
    query_cleaned = ' '.join([word for word in query.split() if word not in stop_words])
    lemmas = m.lemmatize(query_cleaned)
    lemmas1 = ''.join(lemmas)
    query_words = lemmas1.split()
    d = defaultdict(list)
    j = 1
    dl_sum = 0
    texts_dl = []
    for root, dirs, files in os.walk('texts'):
        for fname in files:
            if fname.endswith('.txt'):
                f = open (root + '/' + fname, 'r', encoding = 'utf-8')
                text = f.read()
                lemmast = m.lemmatize(text)
                lemmast1 = ''.join(lemmas)
                words = lemmast1.split()
                for i in range(len(words)):
                    words[i] = words[i].strip('(),./?!|\;:%=+ — "<>[]')
                texts_dl.append(len(words))
                k = Counter(words)
                for word in words:
                    if (j, k[word]) not in d[word]:
                        d[word].append((j, k[word]))
                dl_sum += len(words)
                j+=1
    
    avdl = int(dl_sum/1000)
    itog = defaultdict(float)
    for qw in query_words:
        if qw in d:
            r = d[qw]
            for it in r:
                tx = int(it[0])
                n = int(it[1])
                res = score_BM25(n, len(r), 1000, int(texts_dl[tx-1]), avdl)
                itog[tx]+=res

    b = list(itog.items())
    b.sort(key=lambda item: item[1], reverse=True)
    rang_texts = b[:10]
    links = []
    for txt in rang_texts:
        nazv = str(txt[0])
        f1 = open ('C:/Users/Qwert/Documents/инфопоиск/texts'+ '/' + nazv + '.txt', 'r', encoding = 'utf-8')
        text1 = f1.read()  
        ur = re.findall(r'@url (http://.*?html)', text1)[0]
        if ur:
            links.append((ur, txt[1]))
    return links

@app.route('/', methods=['get','post'])
def index():
    if request.form:
        query = request.form['query']
        links = search_words(query)
        return render_template('index.html',**locals())
    return render_template('index.html')

if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Oct/2017 20:36:44] "GET / HTTP/1.1" 200 -
